In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import *
import pyspark.sql.functions as F
from pyspark.sql.functions import *
from datetime import datetime

In [2]:
spark = SparkSession.builder.master("local").appName("ETL").getOrCreate()

In [3]:
spark

In [4]:
Access_key_ID="*******************"
Secret_access_key="*******************************"
# Enable hadoop s3a settings
spark.sparkContext._jsc.hadoopConfiguration().set("com.amazonaws.services.s3.enableV4", "true")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.aws.credentials.provider", \
                                     "com.amazonaws.auth.InstanceProfileCredentialsProvider,com.amazonaws.auth.DefaultAWSCredentialsProviderChain")
spark.sparkContext._jsc.hadoopConfiguration().set("fs.AbstractFileSystem.s3a.impl", "org.apache.hadoop.fs.s3a.S3A")

spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key",Access_key_ID)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key",Secret_access_key)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.endpoint", "s3.us-east-2.amazonaws.com")

In [5]:
df1=spark.read.format('csv').option("header","true").option("inferSchema","true").load("s3a://staging-area-datalake/1bank_2023-03-18.csv")

In [6]:
df2=spark.read.format('csv').option("header","true").option("inferSchema","true").load("s3a://staging-area-datalake/2bank_2023-03-18.csv")

In [7]:
df1.show()

+---+-------------+-------+---------+--------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+------+
|age|          job|marital|education|default1|balance|housing|loan| contact|day|month|duration|campaign|pdays|previous|poutcome|target|
+---+-------------+-------+---------+--------+-------+-------+----+--------+---+-----+--------+--------+-----+--------+--------+------+
| 30|   unemployed|married|  primary|      no|   1787|     no|  no|cellular| 19|  oct|      79|       1|   -1|       0| unknown|    no|
| 33|     services|married|secondary|      no|   4789|    yes| yes|cellular| 11|  may|     220|       1|  339|       4| failure|    no|
| 35|   management| single| tertiary|      no|   1350|    yes|  no|cellular| 16|  apr|     185|       1|  330|       1| failure|    no|
| 30|   management|married| tertiary|      no|   1476|    yes| yes| unknown|  3|  jun|     199|       4|   -1|       0| unknown|    no|
| 59|  blue-collar|married|secondary|      no|  

In [8]:
df2.show()

+---+------------+--------+---------+--------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+------+
|age|         job| marital|education|default1|balance|housing|loan|  contact|day|month|duration|campaign|pdays|previous|poutcome|target|
+---+------------+--------+---------+--------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+------+
| 34|   housemaid| married| tertiary|      no|  -4057|     no| yes| cellular| 31|  oct|    1068|      47|  358|      19| unknown|   yes|
| 51|  unemployed|divorced|secondary|      no|  42925|     no| yes|telephone| 30|  may|    4585|       9|  505|      11| unknown|   yes|
| 27|   housemaid| married|  primary|      no|  36947|     no| yes| cellular|  7|  jan|    4677|       5|   21|      20| unknown|   yes|
| 35|  unemployed|  single|  primary|      no|  13263|    yes| yes| cellular|  5|  sep|    1964|       2|   87|      19| failure|   yes|
| 44|   housemaid|  single|secondary|    

In [9]:
# Schema of Dataframe 1

df1.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default1: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- target: string (nullable = true)



In [11]:
#schema of Dataframe 2

df2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default1: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- target: string (nullable = true)



In [12]:
# Total Rows of Dataframe 1

df1.count()

4521

In [13]:
# Total Rows of Dataframe 2

df2.count()

550

In [19]:
# Find duplicate rows in Dataframe 1

duplicate_rows = df1.count() - df1.dropDuplicates().count()
print(duplicate_rows)

0


In [20]:
# Find duplicate rows in Dataframe 2

duplicate_rows = df2.count() - df2.dropDuplicates().count()
print(duplicate_rows)

10


In [21]:
# Drop duplicate rows and update the original DataFrame
df2 = df2.dropDuplicates()

In [22]:
df2.count()

540

In [23]:
# Finding Null values in Dataframe 1

df1.select([count(when(isnan(c),c)).alias(c) for c in df1.columns]).show()

+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|age|job|marital|education|default1|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|target|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|  0|  0|      0|        0|       0|      0|      0|   0|      0|  0|    0|       0|       0|    0|       0|       0|     0|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+



In [24]:
null_counts = df1.agg(*[count(when(col(c).isNull(), c)).alias(c) for c in df1.columns])

# Display the null counts
null_counts.show()

+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|age|job|marital|education|default1|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|target|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|  0|  0|      0|        0|       0|      0|      0|   0|      0|  0|    0|       0|       0|    0|       0|       0|     0|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+



In [25]:
# Finding Null values in Dataframe 2

df2.select([count(when(isnan(c),c)).alias(c) for c in df2.columns]).show()

+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|age|job|marital|education|default1|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|target|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|  0|  0|      0|        0|       0|      0|      0|   0|      0|  0|    0|       0|       0|    0|       0|       0|     0|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+



In [26]:
null_counts = df2.agg(*[count(when(col(c).isNull(), c)).alias(c) for c in df2.columns])

# Display the null counts
null_counts.show()

+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|age|job|marital|education|default1|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|target|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|  0|  0|      0|        0|      40|      0|      0|  40|      0|  0|    0|       0|       0|    0|       0|      40|     0|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+



In [29]:
# Calculate mode of the column
mode = df2.groupBy('default1').agg(count('*').alias('count')).orderBy(col('count').desc()).select(col('default1')).first()[0]

# Fill null values with mode value
df2 = df2.fillna(mode, subset=['default1'])

In [31]:
# Calculate mode of the column
mode = df2.groupBy('loan').agg(count('*').alias('count')).orderBy(col('count').desc()).select(col('loan')).first()[0]

# Fill null values with mode value
df2 = df2.fillna(mode, subset=['loan'])

In [32]:
# Calculate mode of the column
mode = df2.groupBy('poutcome').agg(count('*').alias('count')).orderBy(col('count').desc()).select(col('poutcome')).first()[0]

# Fill null values with mode value
df2 = df2.fillna(mode, subset=['poutcome'])

In [33]:
null_counts = df2.agg(*[count(when(col(c).isNull(), c)).alias(c) for c in df2.columns])

# Display the null counts
null_counts.show()

+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|age|job|marital|education|default1|balance|housing|loan|contact|day|month|duration|campaign|pdays|previous|poutcome|target|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+
|  0|  0|      0|        0|       0|      0|      0|   0|      0|  0|    0|       0|       0|    0|       0|       0|     0|
+---+---+-------+---------+--------+-------+-------+----+-------+---+-----+--------+--------+-----+--------+--------+------+



In [34]:
df1.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default1: string (nullable = true)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = true)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = true)
 |-- target: string (nullable = true)



In [35]:
df2.printSchema()

root
 |-- age: integer (nullable = true)
 |-- job: string (nullable = true)
 |-- marital: string (nullable = true)
 |-- education: string (nullable = true)
 |-- default1: string (nullable = false)
 |-- balance: integer (nullable = true)
 |-- housing: string (nullable = true)
 |-- loan: string (nullable = false)
 |-- contact: string (nullable = true)
 |-- day: integer (nullable = true)
 |-- month: string (nullable = true)
 |-- duration: integer (nullable = true)
 |-- campaign: integer (nullable = true)
 |-- pdays: integer (nullable = true)
 |-- previous: integer (nullable = true)
 |-- poutcome: string (nullable = false)
 |-- target: string (nullable = true)



In [37]:
# concatenate the two dataframes

df = df1.union(df2)

In [38]:
#Total Records after joining both dataframe

df.count()

5061

In [41]:
df.show(50,truncate=False)

+---+-------------+--------+---------+--------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+------+
|age|job          |marital |education|default1|balance|housing|loan|contact  |day|month|duration|campaign|pdays|previous|poutcome|target|
+---+-------------+--------+---------+--------+-------+-------+----+---------+---+-----+--------+--------+-----+--------+--------+------+
|30 |unemployed   |married |primary  |no      |1787   |no     |no  |cellular |19 |oct  |79      |1       |-1   |0       |unknown |no    |
|33 |services     |married |secondary|no      |4789   |yes    |yes |cellular |11 |may  |220     |1       |339  |4       |failure |no    |
|35 |management   |single  |tertiary |no      |1350   |yes    |no  |cellular |16 |apr  |185     |1       |330  |1       |failure |no    |
|30 |management   |married |tertiary |no      |1476   |yes    |yes |unknown  |3  |jun  |199     |4       |-1   |0       |unknown |no    |
|59 |blue-collar  |married |second

In [43]:
# Configure Redshift credentials and database information
redshift_jdbc_url = "jdbc:redshift://redshift.clxkpdwlmkpt.us-east-2.redshift.amazonaws.com:5439/dev"
redshift_user = "awsuser"
redshift_password = "AWS$user7"
redshift_database = "dev"
redshift_table = "bank"

In [44]:
# Write DataFrame to Redshift
df.write \
    .format("jdbc") \
    .option("url", redshift_jdbc_url) \
    .option("dbtable", redshift_table) \
    .option("user", redshift_user) \
    .option("password", redshift_password) \
    .option("truncate", "true") \
    .mode("append") \
    .save()